## Setup

In [1]:
!git clone https://github.com/GNNatan/SemNov_AML_DAAI_23-24.git

Cloning into 'SemNov_AML_DAAI_23-24'...
remote: Enumerating objects: 1082, done.
remote: Counting objects: 100% (547/547), done.
remote: Compressing objects: 100% (324/324), done.
remote: Total 1082 (delta 260), reused 426 (delta 212), pack-reused 535
Receiving objects: 100% (1082/1082), 3.26 GiB | 26.33 MiB/s, done.
Resolving deltas: 100% (332/332), done.
Updating files: 100% (739/739), done.


In [1]:
%cd SemNov_AML_DAAI_23-24/

/content/SemNov_AML_DAAI_23-24


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.5/881.5 kB 67.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3

In [2]:
!sh download_data.sh

============Downloading ModelNet40 + OOD Splits in 
--2024-06-13 15:29:35--  https://www.dropbox.com/s/c2x3h59nxprjs21/modelnet40_normal_resampled.tar?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6057:18::a27d:d12
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/c2x3h59nxprjs21/modelnet40_normal_resampled.tar [following]
--2024-06-13 15:29:36--  https://www.dropbox.com/s/dl/c2x3h59nxprjs21/modelnet40_normal_resampled.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc47c810cb1be38cb9ed3c3dbb31.dl.dropboxusercontent.com/cd/0/get/CUzOKrdMC7W4tO-vBfnuEzk6zsod9Viak8vp-nTg02zjlGiLF9bcdVt-WTiwWec_8iGwwt9cJoYalq5Wdv-vcS4fEK_GsJ6NcTDmxaf9l9WDJRtg2WVx3_1hfx4AZTwUw8RCKeHrHJrSMdewpdHSF1wW/file?dl=1# [following]
--2024-06-13 15:29:36--  https://uc47c810cb1be38cb9ed3c3dbb31.dl.dropboxusercontent.com/

In [3]:
import torch
import os
import h5py
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from random import sample
from tqdm import tqdm
from mpl_toolkits.mplot3d import proj3d
from sklearn.metrics import roc_curve
from notebooks.utils import *

In [185]:
SR1 = {
    4: 0,  # chair
    8: 1,  # shelf
    7: 2,  # door
    12: 3,  # sink
    13: 4  # sofa
}

SR2 = {
    10: 0,  # bed
    14: 1,  # toilet
    5: 2,  # desk
    6: 3,  # display
    9: 2  # table
}

SR3 = {
    0: 404,  # bag
    1: 404,  # bin
    2: 404,  # box
    3: 404,  # cabinet
    11: 404  # pillow
}

test_file = h5py.File('3D_OS_release_data/ScanObjectNN/h5_files/main_split/test_objectdataset.h5', 'r')
train_file = h5py.File('3D_OS_release_data/ScanObjectNN/h5_files/main_split/training_objectdataset.h5', 'r')

test_labels  = test_file['label'][:]
train_labels = train_file['label'][:]

test_clouds = test_file['data'][:]
train_clouds = train_file['data'][:]

all_labels = np.hstack((train_labels, test_labels))
all_clouds = np.vstack((train_clouds, test_clouds))

SR1_indices = [index for index, value in enumerate(all_labels) if value in SR1.keys()]
SR2_indices = [index for index, value in enumerate(all_labels) if value in SR2.keys()]
SR3_indices = [index for index, value in enumerate(all_labels) if value in SR3.keys()]

SR1_clouds = all_clouds[SR1_indices]
SR2_clouds = all_clouds[SR2_indices]
SR3_clouds = all_clouds[SR3_indices]

SR1_labels = [SR1[label] for label in all_labels[SR1_indices]]
SR2_labels = [SR2[label] for label in all_labels[SR2_indices]]
SR3_labels = [404 for _ in SR3_indices]

all_indices = np.hstack((SR1_indices, SR2_indices, SR3_indices))
all_labels = all_labels[all_indices]

## PointNet++

### SR1

In [169]:
DATA_NAME = 'PN2_CE_SR1'
SRC = SR1_labels
TAR1 = SR2_labels
TAR2 = SR3_labels
lab_to_text = lab_to_text_sr1

#### Discriminative method

In [170]:
##### DISCRIMINATIVE - setup
src_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/src_logits.pt')
tar1_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_logits.pt')
tar2_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_logits.pt')

src_MSP_scores, src_MSP_pred = F.softmax(src_logits, dim=1).max(1)
tar1_MSP_scores, tar1_MSP_pred = F.softmax(tar1_logits, dim=1).max(1)
tar2_MSP_scores, tar2_MSP_pred = F.softmax(tar2_logits, dim=1).max(1)

In [171]:
# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [172]:
scores_MSP = torch.hstack((src_MSP_scores, tar1_MSP_scores, tar2_MSP_scores))
preds_MSP = torch.hstack((src_MSP_pred, tar1_MSP_pred, tar2_MSP_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [173]:
# ACCURACY_CHECK
acc = 0
for i in range(len(SRC)):
  if truth[i] == preds_MSP[i]:
    acc += 1

print(acc/len(SRC))

0.7880478087649403


In [174]:
# FIND THE THRESHOLD FOR 95 TPR
ood_truth = torch.from_numpy(np.hstack((np.ones_like(src_labels), np.zeros_like(tar1_labels), np.zeros_like(tar2_labels))))

fpr, tpr, thr = roc_curve(ood_truth, scores_MSP, pos_label=1)

for i in range(len(tpr)-1, 0, -1):
  if tpr[i] < 0.95:
    break

print("FPR at 95:", fpr[i])
threshold = thr[i+1]

FPR at 95: 0.8048929663608563


In [175]:
correct = list()
wrong = list()
preds = list()
fp = 0

for i in range(len(truth)):
  pred = preds_MSP[i]
  if scores_MSP[i] < threshold:
    pred = torch.tensor(404)
  if pred == truth[i]:
    correct.append(i)
  else:
    wrong.append(i)
    if truth[i] == torch.tensor(404): # false positive
      fp += 1
  preds.append(pred)

print("Correctly classified", len(correct), "samples.",
      "\nMisclassified", len(wrong), "samples.",
      "\nFPR:", fp/(len(truth) - len(src_labels)))  # sanity check

Correctly classified 1287 samples. 
Misclassified 1603 samples. 
FPR: 0.8048929663608563


In [176]:
OUTPUT_FOLDER = 'outputs/classification_results'
if not os.path.exists(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP'):
  os.makedirs(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP')

for sample in tqdm(range(len(truth))):
  title = f"{lab_to_text[truth[sample].item()]}-{lab_to_text[preds[sample].item()]}"
  file = open(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP/{title}.txt', 'a+')
  file.write(str(sample)+'\n')
  file.close()


100%|██████████| 2890/2890 [00:00<00:00, 11575.80it/s]


#### Distance based method

In [177]:
##### DISTANCE BASED - setup

train_labels = load_from_file(f"outputs/tensors/{DATA_NAME}/train_labels.pt")
src_feats = load_from_file(f'outputs/feats/{DATA_NAME}.pt')

src_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/src_dist.pt')
src_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/src_ids.pt')
tar1_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_dist.pt')
tar1_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_ids.pt')
tar2_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_dist.pt')
tar2_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_ids.pt')

src_L2_dist = src_dist.squeeze().cpu()
src_L2_ids = src_ids.squeeze().cpu()  # index of nearest training sample
src_L2_scores = 1 / src_dist

tar1_L2_dist = tar1_dist.squeeze().cpu()
tar1_L2_ids = tar1_ids.squeeze().cpu()  # index of nearest training sample
tar1_L2_scores = 1 / tar1_dist

tar2_L2_dist = tar2_dist.squeeze().cpu()
tar2_L2_ids = tar2_ids.squeeze().cpu()  # index of nearest training sample
tar2_L2_scores = 1 / tar2_dist


# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [178]:
# train truth
src_L2_pred = torch.from_numpy(train_labels[src_ids])
tar1_L2_pred = torch.from_numpy(train_labels[tar1_ids])
tar2_L2_pred = torch.from_numpy(train_labels[tar2_ids])


In [179]:
scores_L2 = torch.hstack((src_L2_scores, tar1_L2_scores, tar2_L2_scores))
preds_L2 = torch.hstack((src_L2_pred, tar1_L2_pred, tar2_L2_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [180]:
# FIND THE THRESHOLD FOR 95 TPR
ood_truth = torch.from_numpy(np.hstack((np.ones_like(src_labels), np.zeros_like(tar1_labels), np.zeros_like(tar2_labels))))

fpr, tpr, thr = roc_curve(ood_truth, scores_L2, pos_label=1)

for i in range(len(tpr)-1, 0, -1):
  if tpr[i] < 0.95:
    break

print("FPR at 95:", fpr[i])
threshold = thr[i+1]

FPR at 95: 0.8379204892966361


In [181]:
# ACCURACY_CHECK
acc = 0
for i in range(len(SRC)):
  if truth[i] == preds_L2[i]:
    acc += 1

print(acc/len(SRC))

0.7800796812749003


In [182]:
correct = list()
wrong = list()
preds = list()
fp = 0

for i in range(len(truth)):
  pred = preds_L2[i]
  if scores_L2[i] < threshold:
    pred = torch.tensor(404)
  if pred == truth[i]:
    correct.append(i)
  else:
    wrong.append(i)
    if truth[i] == torch.tensor(404): # false positive
      fp += 1
  preds.append(pred)

print("Correctly classified", len(correct), "samples.",
      "\nMisclassified", len(wrong), "samples.",
      "\nFPR:", fp/(len(truth) - len(src_labels)))  # sanity check

Correctly classified 1223 samples. 
Misclassified 1667 samples. 
FPR: 0.8379204892966361


In [183]:
OUTPUT_FOLDER = 'outputs/classification_results'
if not os.path.exists(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2'):
  os.makedirs(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2')

for sample in tqdm(range(len(truth))):
  title = f"{lab_to_text[truth[sample].item()]}-{lab_to_text[preds[sample].item()]}"
  file = open(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2/{title}.txt', 'a+')
  file.write(str(sample)+'\n')
  file.close()

100%|██████████| 2890/2890 [00:00<00:00, 13760.15it/s]


### SR2

In [187]:
DATA_NAME = 'PN2_CE_SR2'
SRC = SR2_labels
TAR1 = SR1_labels
TAR2 = SR3_labels
lab_to_text = lab_to_text_sr2

#### Discriminative method

In [188]:
##### DISCRIMINATIVE - setup
src_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/src_logits.pt')
tar1_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_logits.pt')
tar2_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_logits.pt')

src_MSP_scores, src_MSP_pred = F.softmax(src_logits, dim=1).max(1)
tar1_MSP_scores, tar1_MSP_pred = F.softmax(tar1_logits, dim=1).max(1)
tar2_MSP_scores, tar2_MSP_pred = F.softmax(tar2_logits, dim=1).max(1)

In [189]:
# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [190]:
scores_MSP = torch.hstack((src_MSP_scores, tar1_MSP_scores, tar2_MSP_scores))
preds_MSP = torch.hstack((src_MSP_pred, tar1_MSP_pred, tar2_MSP_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [191]:
# ACCURACY_CHECK
acc = 0
for i in range(len(SRC)):
  if truth[i] == preds_MSP[i]:
    acc += 1

print(acc/len(SRC))

0.8375634517766497


In [192]:
# FIND THE THRESHOLD FOR 95 TPR
ood_truth = torch.from_numpy(np.hstack((np.ones_like(src_labels), np.zeros_like(tar1_labels), np.zeros_like(tar2_labels))))

fpr, tpr, thr = roc_curve(ood_truth, scores_MSP, pos_label=1)

for i in range(len(tpr)-1, 0, -1):
  if tpr[i] < 0.95:
    break

print("FPR at 95:", fpr[i])
threshold = thr[i+1]

FPR at 95: 0.8691722169362512


In [193]:
correct = list()
wrong = list()
preds = list()
fp = 0

for i in range(len(truth)):
  pred = preds_MSP[i]
  if scores_MSP[i] < threshold:
    pred = torch.tensor(404)
  if pred == truth[i]:
    correct.append(i)
  else:
    wrong.append(i)
    if truth[i] == torch.tensor(404): # false positive
      fp += 1
  preds.append(pred)

print("Correctly classified", len(correct), "samples.",
      "\nMisclassified", len(wrong), "samples.",
      "\nFPR:", fp/(len(truth) - len(src_labels)))  # sanity check

Correctly classified 911 samples. 
Misclassified 1979 samples. 
FPR: 0.8691722169362512


In [194]:
OUTPUT_FOLDER = 'outputs/classification_results'
if not os.path.exists(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP'):
  os.makedirs(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP')

for sample in tqdm(range(len(truth))):
  title = f"{lab_to_text[truth[sample].item()]}-{lab_to_text[preds[sample].item()]}"
  file = open(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP/{title}.txt', 'a+')
  file.write(str(sample)+'\n')
  file.close()


100%|██████████| 2890/2890 [00:00<00:00, 18935.53it/s]


#### Distance based method

In [195]:
##### DISTANCE BASED - setup

train_labels = load_from_file(f"outputs/tensors/{DATA_NAME}/train_labels.pt")
src_feats = load_from_file(f'outputs/feats/{DATA_NAME}.pt')

src_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/src_dist.pt')
src_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/src_ids.pt')
tar1_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_dist.pt')
tar1_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_ids.pt')
tar2_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_dist.pt')
tar2_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_ids.pt')

src_L2_dist = src_dist.squeeze().cpu()
src_L2_ids = src_ids.squeeze().cpu()  # index of nearest training sample
src_L2_scores = 1 / src_dist

tar1_L2_dist = tar1_dist.squeeze().cpu()
tar1_L2_ids = tar1_ids.squeeze().cpu()  # index of nearest training sample
tar1_L2_scores = 1 / tar1_dist

tar2_L2_dist = tar2_dist.squeeze().cpu()
tar2_L2_ids = tar2_ids.squeeze().cpu()  # index of nearest training sample
tar2_L2_scores = 1 / tar2_dist


# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [196]:
# train truth
src_L2_pred = torch.from_numpy(train_labels[src_L2_ids])
tar1_L2_pred = torch.from_numpy(train_labels[tar1_L2_ids])
tar2_L2_pred = torch.from_numpy(train_labels[tar2_L2_ids])

In [197]:
scores_L2 = torch.hstack((src_L2_scores, tar1_L2_scores, tar2_L2_scores))
preds_L2 = torch.hstack((src_L2_pred, tar1_L2_pred, tar2_L2_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [198]:
# FIND THE THRESHOLD FOR 95 TPR
ood_truth = torch.from_numpy(np.hstack((np.ones_like(src_labels), np.zeros_like(tar1_labels), np.zeros_like(tar2_labels))))

fpr, tpr, thr = roc_curve(ood_truth, scores_L2, pos_label=1)

for i in range(len(tpr)-1, 0, -1):
  if tpr[i] < 0.95:
    break

print("FPR at 95:", fpr[i])
threshold = thr[i+1]

FPR at 95: 0.807802093244529


In [199]:
# ACCURACY_CHECK
acc = 0
for i in range(len(SRC)):
  if truth[i] == preds_L2[i]:
    acc += 1

print(acc/len(SRC))

0.8362944162436549


In [200]:
correct = list()
wrong = list()
preds = list()
fp = 0

for i in range(len(truth)):
  pred = preds_L2[i]
  if scores_L2[i] < threshold:
    pred = torch.tensor(404)
  if pred == truth[i]:
    correct.append(i)
  else:
    wrong.append(i)
    if truth[i] == torch.tensor(404): # false positive
      fp += 1
  preds.append(pred)

print("Correctly classified", len(correct), "samples.",
      "\nMisclassified", len(wrong), "samples.",
      "\nFPR:", fp/(len(truth) - len(src_labels)))  # sanity check

Correctly classified 1041 samples. 
Misclassified 1849 samples. 
FPR: 0.807802093244529


In [201]:
OUTPUT_FOLDER = 'outputs/classification_results'
if not os.path.exists(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2'):
  os.makedirs(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2')

for sample in tqdm(range(len(truth))):
  title = f"{lab_to_text[truth[sample].item()]}-{lab_to_text[preds[sample].item()]}"
  file = open(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2/{title}.txt', 'a+')
  file.write(str(sample)+'\n')
  file.close()

100%|██████████| 2890/2890 [00:00<00:00, 13512.40it/s]


## DGCNN

### SR1

In [202]:
DATA_NAME = 'DGCNN_CE_SR1'
SRC = SR1_labels
TAR1 = SR2_labels
TAR2 = SR3_labels
lab_to_text = lab_to_text_sr1

#### Discriminative method

In [203]:
##### DISCRIMINATIVE - setup
src_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/src_logits.pt')
tar1_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_logits.pt')
tar2_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_logits.pt')

src_MSP_scores, src_MSP_pred = F.softmax(src_logits, dim=1).max(1)
tar1_MSP_scores, tar1_MSP_pred = F.softmax(tar1_logits, dim=1).max(1)
tar2_MSP_scores, tar2_MSP_pred = F.softmax(tar2_logits, dim=1).max(1)

In [204]:
# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [205]:
scores_MSP = torch.hstack((src_MSP_scores, tar1_MSP_scores, tar2_MSP_scores))
preds_MSP = torch.hstack((src_MSP_pred, tar1_MSP_pred, tar2_MSP_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [206]:
# ACCURACY_CHECK
acc = 0
for i in range(len(SRC)):
  if truth[i] == preds_MSP[i]:
    acc += 1

print(acc/len(SRC))

0.7243027888446215


In [207]:
# FIND THE THRESHOLD FOR 95 TPR
ood_truth = torch.from_numpy(np.hstack((np.ones_like(src_labels), np.zeros_like(tar1_labels), np.zeros_like(tar2_labels))))

fpr, tpr, thr = roc_curve(ood_truth, scores_MSP, pos_label=1)

for i in range(len(tpr)-1, 0, -1):
  if tpr[i] < 0.95:
    break

print("FPR at 95:", fpr[i])
threshold = thr[i+1]

FPR at 95: 0.9015290519877676


In [208]:
correct = list()
wrong = list()
preds = list()
fp = 0

for i in range(len(truth)):
  pred = preds_MSP[i]
  if scores_MSP[i] < threshold:
    pred = torch.tensor(404)
  if pred == truth[i]:
    correct.append(i)
  else:
    wrong.append(i)
    if truth[i] == torch.tensor(404): # false positive
      fp += 1
  preds.append(pred)

print("Correctly classified", len(correct), "samples.",
      "\nMisclassified", len(wrong), "samples.",
      "\nFPR:", fp/(len(truth) - len(src_labels)))  # sanity check

Correctly classified 1047 samples. 
Misclassified 1843 samples. 
FPR: 0.9015290519877676


In [209]:
OUTPUT_FOLDER = 'outputs/classification_results'
if not os.path.exists(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP'):
  os.makedirs(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP')

for sample in tqdm(range(len(truth))):
  title = f"{lab_to_text[truth[sample].item()]}-{lab_to_text[preds[sample].item()]}"
  file = open(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP/{title}.txt', 'a+')
  file.write(str(sample)+'\n')
  file.close()


100%|██████████| 2890/2890 [00:00<00:00, 14038.92it/s]


#### Distance based method

In [210]:
##### DISTANCE BASED - setup

train_labels = load_from_file(f"outputs/tensors/{DATA_NAME}/train_labels.pt")
src_feats = load_from_file(f'outputs/feats/{DATA_NAME}.pt')

src_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/src_dist.pt')
src_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/src_ids.pt')
tar1_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_dist.pt')
tar1_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_ids.pt')
tar2_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_dist.pt')
tar2_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_ids.pt')

src_L2_dist = src_dist.squeeze().cpu()
src_L2_ids = src_ids.squeeze().cpu()  # index of nearest training sample
src_L2_scores = 1 / src_dist

tar1_L2_dist = tar1_dist.squeeze().cpu()
tar1_L2_ids = tar1_ids.squeeze().cpu()  # index of nearest training sample
tar1_L2_scores = 1 / tar1_dist

tar2_L2_dist = tar2_dist.squeeze().cpu()
tar2_L2_ids = tar2_ids.squeeze().cpu()  # index of nearest training sample
tar2_L2_scores = 1 / tar2_dist


# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [211]:
# train truth
src_L2_pred = torch.from_numpy(train_labels[src_ids])
tar1_L2_pred = torch.from_numpy(train_labels[tar1_ids])
tar2_L2_pred = torch.from_numpy(train_labels[tar2_ids])


In [212]:
scores_L2 = torch.hstack((src_L2_scores, tar1_L2_scores, tar2_L2_scores))
preds_L2 = torch.hstack((src_L2_pred, tar1_L2_pred, tar2_L2_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [213]:
# FIND THE THRESHOLD FOR 95 TPR
ood_truth = torch.from_numpy(np.hstack((np.ones_like(src_labels), np.zeros_like(tar1_labels), np.zeros_like(tar2_labels))))

fpr, tpr, thr = roc_curve(ood_truth, scores_L2, pos_label=1)

for i in range(len(tpr)-1, 0, -1):
  if tpr[i] < 0.95:
    break

print("FPR at 95:", fpr[i])
threshold = thr[i+1]

FPR at 95: 0.8538226299694189


In [214]:
# ACCURACY_CHECK
acc = 0
for i in range(len(SRC)):
  if truth[i] == preds_L2[i]:
    acc += 1

print(acc/len(SRC))

0.7298804780876494


In [215]:
correct = list()
wrong = list()
preds = list()
fp = 0

for i in range(len(truth)):
  pred = preds_L2[i]
  if scores_L2[i] < threshold:
    pred = torch.tensor(404)
  if pred == truth[i]:
    correct.append(i)
  else:
    wrong.append(i)
    if truth[i] == torch.tensor(404): # false positive
      fp += 1
  preds.append(pred)

print("Correctly classified", len(correct), "samples.",
      "\nMisclassified", len(wrong), "samples.",
      "\nFPR:", fp/(len(truth) - len(src_labels)))  # sanity check

Correctly classified 1125 samples. 
Misclassified 1765 samples. 
FPR: 0.8538226299694189


In [216]:
OUTPUT_FOLDER = 'outputs/classification_results'
if not os.path.exists(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2'):
  os.makedirs(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2')

for sample in tqdm(range(len(truth))):
  title = f"{lab_to_text[truth[sample].item()]}-{lab_to_text[preds[sample].item()]}"
  file = open(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2/{title}.txt', 'a+')
  file.write(str(sample)+'\n')
  file.close()

100%|██████████| 2890/2890 [00:00<00:00, 13758.73it/s]


### SR2

In [217]:
DATA_NAME = 'DGCNN_CE_SR2'
SRC = SR2_labels
TAR1 = SR1_labels
TAR2 = SR3_labels
lab_to_text = lab_to_text_sr2

#### Discriminative method

In [218]:
##### DISCRIMINATIVE - setup
src_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/src_logits.pt')
tar1_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_logits.pt')
tar2_logits = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_logits.pt')

src_MSP_scores, src_MSP_pred = F.softmax(src_logits, dim=1).max(1)
tar1_MSP_scores, tar1_MSP_pred = F.softmax(tar1_logits, dim=1).max(1)
tar2_MSP_scores, tar2_MSP_pred = F.softmax(tar2_logits, dim=1).max(1)

In [219]:
# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [220]:
scores_MSP = torch.hstack((src_MSP_scores, tar1_MSP_scores, tar2_MSP_scores))
preds_MSP = torch.hstack((src_MSP_pred, tar1_MSP_pred, tar2_MSP_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [221]:
# ACCURACY_CHECK
acc = 0
for i in range(len(SRC)):
  if truth[i] == preds_MSP[i]:
    acc += 1

print(acc/len(SRC))

0.6789340101522843


In [222]:
# FIND THE THRESHOLD FOR 95 TPR
ood_truth = torch.from_numpy(np.hstack((np.ones_like(src_labels), np.zeros_like(tar1_labels), np.zeros_like(tar2_labels))))

fpr, tpr, thr = roc_curve(ood_truth, scores_MSP, pos_label=1)

for i in range(len(tpr)-1, 0, -1):
  if tpr[i] < 0.95:
    break

print("FPR at 95:", fpr[i])
threshold = thr[i+1]

FPR at 95: 0.9005708848715509


In [223]:
correct = list()
wrong = list()
preds = list()
fp = 0

for i in range(len(truth)):
  pred = preds_MSP[i]
  if scores_MSP[i] < threshold:
    pred = torch.tensor(404)
  if pred == truth[i]:
    correct.append(i)
  else:
    wrong.append(i)
    if truth[i] == torch.tensor(404): # false positive
      fp += 1
  preds.append(pred)

print("Correctly classified", len(correct), "samples.",
      "\nMisclassified", len(wrong), "samples.",
      "\nFPR:", fp/(len(truth) - len(src_labels)))  # sanity check

Correctly classified 731 samples. 
Misclassified 2159 samples. 
FPR: 0.9005708848715509


In [224]:
OUTPUT_FOLDER = 'outputs/classification_results'
if not os.path.exists(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP'):
  os.makedirs(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP')

for sample in tqdm(range(len(truth))):
  title = f"{lab_to_text[truth[sample].item()]}-{lab_to_text[preds[sample].item()]}"
  file = open(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP/{title}.txt', 'a+')
  file.write(str(sample)+'\n')
  file.close()


100%|██████████| 2890/2890 [00:00<00:00, 14025.95it/s]


#### Distance based method

In [225]:
##### DISTANCE BASED - setup

train_labels = load_from_file(f"outputs/tensors/{DATA_NAME}/train_labels.pt")
src_feats = load_from_file(f'outputs/feats/{DATA_NAME}.pt')

src_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/src_dist.pt')
src_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/src_ids.pt')
tar1_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_dist.pt')
tar1_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_ids.pt')
tar2_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_dist.pt')
tar2_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_ids.pt')

src_L2_dist = src_dist.squeeze().cpu()
src_L2_ids = src_ids.squeeze().cpu()  # index of nearest training sample
src_L2_scores = 1 / src_dist

tar1_L2_dist = tar1_dist.squeeze().cpu()
tar1_L2_ids = tar1_ids.squeeze().cpu()  # index of nearest training sample
tar1_L2_scores = 1 / tar1_dist

tar2_L2_dist = tar2_dist.squeeze().cpu()
tar2_L2_ids = tar2_ids.squeeze().cpu()  # index of nearest training sample
tar2_L2_scores = 1 / tar2_dist


# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [226]:
# train truth
src_L2_pred = torch.from_numpy(train_labels[src_ids])
tar1_L2_pred = torch.from_numpy(train_labels[tar1_ids])
tar2_L2_pred = torch.from_numpy(train_labels[tar2_ids])


In [227]:
scores_L2 = torch.hstack((src_L2_scores, tar1_L2_scores, tar2_L2_scores))
preds_L2 = torch.hstack((src_L2_pred, tar1_L2_pred, tar2_L2_pred))
preds_L2 = torch.where(preds_L2==4, torch.tensor(2), preds_L2)
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [228]:
# FIND THE THRESHOLD FOR 95 TPR
ood_truth = torch.from_numpy(np.hstack((np.ones_like(src_labels), np.zeros_like(tar1_labels), np.zeros_like(tar2_labels))))

fpr, tpr, thr = roc_curve(ood_truth, scores_L2, pos_label=1)

for i in range(len(tpr)-1, 0, -1):
  if tpr[i] < 0.95:
    break

print("FPR at 95:", fpr[i])
threshold = thr[i+1]

FPR at 95: 0.8920076117982874


In [229]:
# ACCURACY_CHECK
acc = 0
for i in range(len(SRC)):
  if truth[i] == preds_L2[i]:
    acc += 1

print(acc/len(SRC))

0.6979695431472082


In [230]:
correct = list()
wrong = list()
preds = list()
fp = 0

for i in range(len(truth)):
  pred = preds_L2[i]
  if scores_L2[i] < threshold:
    pred = torch.tensor(404)
  if pred == truth[i]:
    correct.append(i)
  else:
    wrong.append(i)
    if truth[i] == torch.tensor(404): # false positive
      fp += 1
  preds.append(pred)

print("Correctly classified", len(correct), "samples.",
      "\nMisclassified", len(wrong), "samples.",
      "\nFPR:", fp/(len(truth) - len(src_labels)))  # sanity check

Correctly classified 754 samples. 
Misclassified 2136 samples. 
FPR: 0.8920076117982874


In [231]:
OUTPUT_FOLDER = 'outputs/classification_results'
if not os.path.exists(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2'):
  os.makedirs(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2')

for sample in tqdm(range(len(truth))):
  title = f"{lab_to_text[truth[sample].item()]}-{lab_to_text[preds[sample].item()]}"
  file = open(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2/{title}.txt', 'a+')
  file.write(str(sample)+'\n')
  file.close()

100%|██████████| 2890/2890 [00:00<00:00, 16237.00it/s]


## OpenShape

### SR1

In [232]:
DATA_NAME = 'OpenShape_SR1'
SRC = SR1_labels
TAR1 = SR2_labels
TAR2 = SR3_labels
SRC_indices = SR1_indices
TAR1_indices = SR2_indices
TAR2_indices = SR3_indices
lab_to_text = lab_to_text_sr1

#### Discriminative method

In [233]:
##### DISCRIMINATIVE - setup
img_scores = load_from_file(f'outputs/tensors/{DATA_NAME}/img_scores.pt')
txt_scores = load_from_file(f'outputs/tensors/{DATA_NAME}/text_scores.pt')

openshape_scores = img_scores + txt_scores

src_logits = torch.tensor(np.array([openshape_scores[i] for i in SRC_indices])).squeeze(1)
tar1_logits = torch.tensor(np.array([openshape_scores[i] for i in TAR1_indices])).squeeze(1)
tar2_logits = torch.tensor(np.array([openshape_scores[i] for i in TAR2_indices])).squeeze(1)

src_MSP_scores, src_MSP_pred = src_logits.max(1)
tar1_MSP_scores, tar1_MSP_pred = tar1_logits.max(1)
tar2_MSP_scores, tar2_MSP_pred = tar2_logits.max(1)

In [234]:
# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [235]:
scores_MSP = torch.hstack((src_MSP_scores, tar1_MSP_scores, tar2_MSP_scores))
preds_MSP = torch.hstack((src_MSP_pred, tar1_MSP_pred, tar2_MSP_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [236]:
# ACCURACY_CHECK
acc = 0
for i in range(len(SRC)):
  if truth[i] == preds_MSP[i]:
    acc += 1

print(acc/len(SRC))

0.8589641434262948


In [237]:
# FIND THE THRESHOLD FOR 95 TPR
ood_truth = torch.from_numpy(np.hstack((np.ones_like(src_labels), np.zeros_like(tar1_labels), np.zeros_like(tar2_labels))))

fpr, tpr, thr = roc_curve(ood_truth, scores_MSP, pos_label=1)

for i in range(len(tpr)-1, 0, -1):
  if tpr[i] < 0.95:
    break

print("FPR at 95:", fpr[i])
threshold = thr[i+1]

FPR at 95: 0.6709480122324158


In [238]:
correct = list()
wrong = list()
preds = list()
fp = 0

for i in range(len(truth)):
  pred = preds_MSP[i]
  if scores_MSP[i] < threshold:
    pred = torch.tensor(404)
  if pred == truth[i]:
    correct.append(i)
  else:
    wrong.append(i)
    if truth[i] == torch.tensor(404): # false positive
      fp += 1
  preds.append(pred)

print("Correctly classified", len(correct), "samples.",
      "\nMisclassified", len(wrong), "samples.",
      "\nFPR:", fp/(len(truth) - len(src_labels)))  # sanity check

Correctly classified 1583 samples. 
Misclassified 1307 samples. 
FPR: 0.6709480122324158


In [239]:
OUTPUT_FOLDER = 'outputs/classification_results'
if not os.path.exists(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP'):
  os.makedirs(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP')

for sample in tqdm(range(len(truth))):
  title = f"{lab_to_text[truth[sample].item()]}-{lab_to_text[preds[sample].item()]}"
  file = open(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP/{title}.txt', 'a+')
  file.write(str(sample)+'\n')
  file.close()

100%|██████████| 2890/2890 [00:00<00:00, 23189.57it/s]


#### Distance based method

In [240]:
##### DISTANCE BASED - setup

train_labels = load_from_file(f"outputs/tensors/{DATA_NAME}/train_labels.pt")

src_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/src_dist.pt')
src_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/src_ids.pt')
tar1_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_dist.pt')
tar1_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_ids.pt')
tar2_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_dist.pt')
tar2_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_ids.pt')

src_L2_dist = src_dist.squeeze().cpu()
src_L2_ids = src_ids.squeeze().cpu()  # index of nearest training sample
src_L2_scores = 1 / src_dist

tar1_L2_dist = tar1_dist.squeeze().cpu()
tar1_L2_ids = tar1_ids.squeeze().cpu()  # index of nearest training sample
tar1_L2_scores = 1 / tar1_dist

tar2_L2_dist = tar2_dist.squeeze().cpu()
tar2_L2_ids = tar2_ids.squeeze().cpu()  # index of nearest training sample
tar2_L2_scores = 1 / tar2_dist


# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [241]:
# train truth
src_L2_pred = torch.from_numpy(train_labels[src_ids])
tar1_L2_pred = torch.from_numpy(train_labels[tar1_ids])
tar2_L2_pred = torch.from_numpy(train_labels[tar2_ids])


In [242]:
scores_L2 = torch.hstack((src_L2_scores, tar1_L2_scores, tar2_L2_scores))
preds_L2 = torch.hstack((src_L2_pred, tar1_L2_pred, tar2_L2_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [243]:
# FIND THE THRESHOLD FOR 95 TPR
ood_truth = torch.from_numpy(np.hstack((np.ones_like(src_labels), np.zeros_like(tar1_labels), np.zeros_like(tar2_labels))))

fpr, tpr, thr = roc_curve(ood_truth, scores_L2, pos_label=1)

for i in range(len(tpr)-1, 0, -1):
  if tpr[i] < 0.95:
    break

print("FPR at 95:", fpr[i])
threshold = thr[i+1]

FPR at 95: 0.8159021406727829


In [244]:
# ACCURACY_CHECK
acc = 0
for i in range(len(SRC)):
  if truth[i] == preds_L2[i]:
    acc += 1

print(acc/len(SRC))

0.9139442231075697


In [245]:
correct = list()
wrong = list()
preds = list()
fp = 0

for i in range(len(truth)):
  pred = preds_L2[i]
  if scores_L2[i] < threshold:
    pred = torch.tensor(404)
  if pred == truth[i]:
    correct.append(i)
  else:
    wrong.append(i)
    if truth[i] == torch.tensor(404): # false positive
      fp += 1
  preds.append(pred)

print("Correctly classified", len(correct), "samples.",
      "\nMisclassified", len(wrong), "samples.",
      "\nFPR:", fp/(len(truth) - len(src_labels)))  # sanity check

Correctly classified 1401 samples. 
Misclassified 1489 samples. 
FPR: 0.8159021406727829


In [246]:
OUTPUT_FOLDER = 'outputs/classification_results'
if not os.path.exists(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2'):
  os.makedirs(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2')

for sample in tqdm(range(len(truth))):
  title = f"{lab_to_text[truth[sample].item()]}-{lab_to_text[preds[sample].item()]}"
  file = open(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2/{title}.txt', 'a+')
  file.write(str(sample)+'\n')
  file.close()

100%|██████████| 2890/2890 [00:00<00:00, 24494.14it/s]


### SR2

In [247]:
DATA_NAME = 'OpenShape_SR2'
SRC = SR2_labels
TAR1 = SR1_labels
TAR2 = SR3_labels
SRC_indices = SR2_indices
TAR1_indices = SR1_indices
TAR2_indices = SR3_indices
lab_to_text = lab_to_text_sr2

#### Discriminative method

In [248]:
##### DISCRIMINATIVE - setup
img_scores = load_from_file(f'outputs/tensors/{DATA_NAME}/img_scores.pt')
txt_scores = load_from_file(f'outputs/tensors/{DATA_NAME}/text_scores.pt')

openshape_scores = img_scores + txt_scores

src_logits = torch.tensor(np.array([openshape_scores[i] for i in SRC_indices])).squeeze(1)
tar1_logits = torch.tensor(np.array([openshape_scores[i] for i in TAR1_indices])).squeeze(1)
tar2_logits = torch.tensor(np.array([openshape_scores[i] for i in TAR2_indices])).squeeze(1)

# merge desk and table prediction into same class for SR2
src_table = src_logits[:, 2] + src_logits[:, 4]
tar1_table = tar1_logits[:, 2] + tar1_logits[:, 4]
tar2_table = tar2_logits[:, 2] + tar2_logits[:, 4]

src_logits = torch.cat((src_logits[:, :2], src_table.unsqueeze(1), src_logits[:, 3].unsqueeze(1)), dim=1)
tar1_logits = torch.cat((tar1_logits[:, :2], tar1_table.unsqueeze(1), tar1_logits[:, 3].unsqueeze(1)), dim=1)
tar2_logits = torch.cat((tar2_logits[:, :2], tar2_table.unsqueeze(1), tar2_logits[:, 3].unsqueeze(1)), dim=1)

src_MSP_scores, src_MSP_pred = src_logits.max(1)
tar1_MSP_scores, tar1_MSP_pred = tar1_logits.max(1)
tar2_MSP_scores, tar2_MSP_pred = tar2_logits.max(1)

In [249]:
# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [250]:
scores_MSP = torch.hstack((src_MSP_scores, tar1_MSP_scores, tar2_MSP_scores))
preds_MSP = torch.hstack((src_MSP_pred, tar1_MSP_pred, tar2_MSP_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [251]:
# ACCURACY_CHECK
acc = 0
for i in range(len(SRC)):
  if truth[i] == preds_MSP[i]:
    acc += 1

print(acc/len(SRC))

0.5203045685279187


In [252]:
# FIND THE THRESHOLD FOR 95 TPR
ood_truth = torch.from_numpy(np.hstack((np.ones_like(src_labels), np.zeros_like(tar1_labels), np.zeros_like(tar2_labels))))

fpr, tpr, thr = roc_curve(ood_truth, scores_MSP, pos_label=1)

for i in range(len(tpr)-1, 0, -1):
  if tpr[i] < 0.95:
    break

print("FPR at 95:", fpr[i])
threshold = thr[i+1]

FPR at 95: 0.9533777354900095


In [253]:
correct = list()
wrong = list()
preds = list()
fp = 0

for i in range(len(truth)):
  pred = preds_MSP[i]
  if scores_MSP[i] < threshold:
    pred = torch.tensor(404)
  if pred == truth[i]:
    correct.append(i)
  else:
    wrong.append(i)
    if truth[i] == torch.tensor(404): # false positive
      fp += 1
  preds.append(pred)

print("Correctly classified", len(correct), "samples.",
      "\nMisclassified", len(wrong), "samples.",
      "\nFPR:", fp/(len(truth) - len(src_labels)))  # sanity check

Correctly classified 501 samples. 
Misclassified 2389 samples. 
FPR: 0.9533777354900095


In [254]:
OUTPUT_FOLDER = 'outputs/classification_results'
if not os.path.exists(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP'):
  os.makedirs(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP')

for sample in tqdm(range(len(truth))):
  title = f"{lab_to_text[truth[sample].item()]}-{lab_to_text[preds[sample].item()]}"
  file = open(f'{OUTPUT_FOLDER}/{DATA_NAME}/MSP/{title}.txt', 'a+')
  file.write(str(sample)+'\n')
  file.close()

100%|██████████| 2890/2890 [00:00<00:00, 22692.20it/s]


#### Distance based method

In [255]:
##### DISTANCE BASED - setup

train_labels = load_from_file(f"outputs/tensors/{DATA_NAME}/train_labels.pt")

src_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/src_dist.pt')
src_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/src_ids.pt')
tar1_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_dist.pt')
tar1_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar1_ids.pt')
tar2_dist = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_dist.pt')
tar2_ids = load_from_file(f'outputs/tensors/{DATA_NAME}/tar2_ids.pt')

src_L2_dist = src_dist.squeeze().cpu()
src_L2_ids = src_ids.squeeze().cpu()  # index of nearest training sample
src_L2_scores = 1 / src_dist

tar1_L2_dist = tar1_dist.squeeze().cpu()
tar1_L2_ids = tar1_ids.squeeze().cpu()  # index of nearest training sample
tar1_L2_scores = 1 / tar1_dist

tar2_L2_dist = tar2_dist.squeeze().cpu()
tar2_L2_ids = tar2_ids.squeeze().cpu()  # index of nearest training sample
tar2_L2_scores = 1 / tar2_dist


# set OOD labels/truth
src_labels = torch.tensor(SRC)
tar1_labels = torch.tensor(np.full_like(TAR1, 404))
tar2_labels = torch.tensor(np.full_like(TAR2, 404))

In [256]:
# train truth
src_L2_pred = torch.from_numpy(train_labels[src_ids])
tar1_L2_pred = torch.from_numpy(train_labels[tar1_ids])
tar2_L2_pred = torch.from_numpy(train_labels[tar2_ids])


In [257]:
scores_L2 = torch.hstack((src_L2_scores, tar1_L2_scores, tar2_L2_scores))
preds_L2 = torch.hstack((src_L2_pred, tar1_L2_pred, tar2_L2_pred))
truth = torch.hstack((src_labels, tar1_labels, tar2_labels))

In [258]:
# FIND THE THRESHOLD FOR 95 TPR
ood_truth = torch.from_numpy(np.hstack((np.ones_like(src_labels), np.zeros_like(tar1_labels), np.zeros_like(tar2_labels))))

fpr, tpr, thr = roc_curve(ood_truth, scores_L2, pos_label=1)

for i in range(len(tpr)-1, 0, -1):
  if tpr[i] < 0.95:
    break

print("FPR at 95:", fpr[i])
threshold = thr[i+1]

FPR at 95: 0.8810656517602283


In [259]:
# ACCURACY_CHECK
acc = 0
for i in range(len(SRC)):
  if truth[i] == preds_L2[i]:
    acc += 1

print(acc/len(SRC))

0.5647208121827412


In [260]:
correct = list()
wrong = list()
preds = list()
fp = 0

for i in range(len(truth)):
  pred = preds_L2[i]
  if scores_L2[i] < threshold:
    pred = torch.tensor(404)
  if pred == truth[i]:
    correct.append(i)
  else:
    wrong.append(i)
    if truth[i] == torch.tensor(404): # false positive
      fp += 1
  preds.append(pred)

print("Correctly classified", len(correct), "samples.",
      "\nMisclassified", len(wrong), "samples.",
      "\nFPR:", fp/(len(truth) - len(src_labels)))  # sanity check

Correctly classified 675 samples. 
Misclassified 2215 samples. 
FPR: 0.8810656517602283


In [261]:
OUTPUT_FOLDER = 'outputs/classification_results'
if not os.path.exists(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2'):
  os.makedirs(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2')

for sample in tqdm(range(len(truth))):
  title = f"{lab_to_text[truth[sample].item()]}-{lab_to_text[preds[sample].item()]}"
  file = open(f'{OUTPUT_FOLDER}/{DATA_NAME}/L2/{title}.txt', 'a+')
  file.write(str(sample)+'\n')
  file.close()

100%|██████████| 2890/2890 [00:00<00:00, 18458.06it/s]
